In [ ]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import re

import logging
logger = logging.getLogger('SPARK')
logger.setLevel(level=logging.WARNING)

import altair as alt
alt.data_transformers.disable_max_rows()

from src.data.query_snowflake import update_week_extremes, read_DALI_week_extremes, read_meta
from src.plot.altair import plot_limits, plot_history
from src.plot.formatting import format_limits, format_history

In [ ]:
# update_week_extremes()

In [ ]:
# # sample_box = "EMD.801267-1"
# df_meta_full = read_meta()

In [ ]:
df_meta_full.query('boxid=="TBB.581265-1"')

In [ ]:
sample_box = df_meta_full.sample().iloc[0,0]
sample_box

In [ ]:
%%time
df_meta = read_meta(sample_box)
df_limits = format_limits(df_meta=df_meta)
df_data = read_DALI_week_extremes(boxid=sample_box, L="sumli")
df_plot = format_history(df_data)
plot_limits(df_limits) + plot_history(df_plot)